# Tratamento prévio dos dados - precipitação no Brasil em (mm)

## Pacotes e importação

In [1]:
import os
import pandas as pd
import numpy as np
import csv
import tqdm
import datetime

In [2]:
# Pegando todos os arquivos baixados do site do INMET.
pasta = './01-dados'
arquivos_nome = []
for diretorio, subpastas, arquivos in os.walk(pasta):
    for arquivo in arquivos:
        #print(arquivo)
        arquivos_nome.append(arquivo)

In [3]:
print('Quantidade de arquivos para tratamento:')
print(len(arquivos_nome))
print('Os primeiro arquivos tem o nome de:')
arquivos_nome[0:5]

Quantidade de arquivos para tratamento:
160
Os primeiro arquivos tem o nome de:


['dados_A501_M_2002-11-03_2008-06-24.csv',
 'dados_A502_M_2002-12-04_2022-01-01.csv',
 'dados_A503_M_2002-12-05_2010-09-21.csv',
 'dados_A504_M_2002-12-08_2004-11-17.csv',
 'dados_A505_M_2002-12-18_2022-01-01.csv']

## Tratamento de dados

In [4]:
dados = []
localidades = []
for i in tqdm.tqdm(arquivos_nome):
    file = open(f'01-dados/{i}')
    #type(file)
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)
    localidades.append(header[0][6:])
    rows = []
    for row in csvreader:
        rows.append(row)
        dado = pd.DataFrame(rows[10:])
        if len(dado)>2:
            dado.columns = ['ttt']
            dado = dado.ttt.str.split(';', 1, expand=True)
            dado.columns = ['data','valor']
            dado['valor'] = dado.valor.str.replace(';','')
            dados.append(dado)
    file.close()

100%|██████████| 160/160 [00:48<00:00,  3.33it/s]


In [5]:
%%time
df = pd.concat(dados)
df = df.dropna()

Wall time: 4.69 s


In [6]:
%%time
df.columns = ['Data','Precipitação (mm)']
df

Wall time: 1.52 s


,Data,Precipitação (mm)
0,2002-11-30,null
1,2002-12-31,null
2,2003-01-31,null
0,2002-11-30,null
1,2002-12-31,null
...,...,...
93,2021-09-30,20.6
94,2021-10-31,246.2
95,2021-11-30,260.8
96,2021-12-31,395.2


In [7]:
%%time
df['Precipitação (mm)'] = pd.to_numeric(df['Precipitação (mm)'], errors='coerce')
df = df.dropna()
df

Wall time: 2.32 s


,Data,Precipitação (mm)
3,2003-02-28,136.8
3,2003-02-28,136.8
4,2003-03-31,113.0
3,2003-02-28,136.8
4,2003-03-31,113.0
...,...,...
92,2021-08-31,23.6
93,2021-09-30,20.6
94,2021-10-31,246.2
95,2021-11-30,260.8


In [8]:
%%time
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')
df['Mês'] = df['Data'].dt.month
df['Ano'] = df['Data'].dt.year
df

<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 845 ms


<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Data,Precipitação (mm),Mês,Ano
3,2003-02-28,136.8,2,2003
3,2003-02-28,136.8,2,2003
4,2003-03-31,113.0,3,2003
3,2003-02-28,136.8,2,2003
4,2003-03-31,113.0,3,2003
...,...,...,...,...
92,2021-08-31,23.6,8,2021
93,2021-09-30,20.6,9,2021
94,2021-10-31,246.2,10,2021
95,2021-11-30,260.8,11,2021


In [9]:
resumo = df.groupby(['Ano','Mês']).agg({'Precipitação (mm)':['mean','median','max','min']})
resumo = resumo.round(2)
resumo = resumo.reset_index()
resumo

Ano Mês Precipitação (mm)                     
                           mean median    max    min
0    2000   6              2.60    2.6    2.6    2.6
1    2000   7             77.00   77.0   77.0   77.0
2    2000  10             57.00   57.0   57.0   57.0
3    2000  11            161.00  161.0  161.0  161.0
4    2001   3            122.60  122.6  122.6  122.6
..    ...  ..               ...    ...    ...    ...
234  2021   8             25.94   12.4  174.6    0.0
235  2021   9             25.38   20.6  121.4    0.0
236  2021  10            190.30  189.1  367.2   21.4
237  2021  11            186.40  180.6  518.6   40.2
238  2021  12            263.68  272.5  579.0    0.0

[239 rows x 6 columns]

In [10]:
locais = pd.DataFrame(localidades)
locais.columns = 'Cidades'
locais

TypeError: Index(...) must be called with a collection of some kind, 'Cidades' was passed

In [ ]:
resumo.hist()

## Exportando as informações

In [ ]:
resumo.to_excel('resumo_precipitação.xlsx')
resumo.to_csv('resumo_precipitação.csv', index=False)
locais.to_excel('cidades_em_análise.xlsx')